# Extract TOMT Submissions

In [1]:
from pyspark import SparkConf, SparkContext
from glob import glob
import sys
import json
import io
from tqdm import tqdm
import bz2

REDDIT_SUBMISSION_DIR = '/mnt/ceph/storage/corpora/corpora-thirdparty/corpus-reddit-submissions/'
DATA_DIR = '/mnt/ceph/storage/data-in-progress/data-research/web-search/false-memories/'
SUBREDDIT = 'tipofmytongue'

conf=SparkConf()
conf.setAll([("spark.executor.memory", '20g'), ("spark.driver.memory", '20g')])
sc = SparkContext(master="yarn",appName="reddit-tomt",conf=conf)
sc

<SparkContext master=yarn appName=reddit-tomt>

In [9]:
def load_zstd_tomt_entries_from_file(file_name):
    zstandard_lib_path = f'{DATA_DIR}reddit-tomt/virtual-env/lib/python3.8/site-packages/'
    if zstandard_lib_path not in sys.path:
        sys.path.insert(0, zstandard_lib_path)
    import zstandard

    with open(file_name,"rb") as inf:
        dctx = zstandard.ZstdDecompressor(max_window_size=2147483648)
        stream_reader = dctx.stream_reader(inf, read_size=16384)
        text_stream = io.TextIOWrapper(stream_reader, encoding='utf-8')
        for idx, line in enumerate(tqdm(text_stream)):
            try:
                record = json.loads(line)
                if record['subreddit'].lower() == SUBREDDIT and record['selftext'] not in ('[removed]', '[deleted]'):
                    yield line
            except:
                pass


In [ ]:
sc.parallelize(glob(f'{REDDIT_SUBMISSION_DIR}/*.zst'))\
    .flatMap(lambda i: load_zstd_tomt_entries_from_file(i))\
    .repartition(100)\
    .saveAsTextFile('reddit-tomt/zstd-files', compressionCodecClass="org.apache.hadoop.io.compress.GzipCodec")